In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName ("trip_count_sql").getOrCreate()
# spark instance 생성 

22/10/04 20:31:04 WARN Utils: Your hostname, Moon-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.4 instead (on interface en0)
22/10/04 20:31:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/04 20:31:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
directory = "/Users/sig6774/Desktop/Data_Engineering/data-engineering-main/01-spark/data"
filename = "tripdata_2021_1-7.csv"

In [6]:
data = spark.read.csv(f"file:///{directory}/{filename}", inferSchema = True, header = True)
# 사전에 정의한 내용으로 csv파일을 읽어올 수 있음 
# inferSchema : 자동으로 스키마를 추론할건지 
# header : header 가지고 올 껀지 
# DataFrame이 생김 

In [7]:
data.show(5)
# head()랑 비슷하다고 생각하면 됨 

22/10/04 20:31:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, airport_fee
 Schema: _c0, VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, airport_fee
Expected: _c0 but found: 
CSV file: file:///Users/sig6774/Desktop/Data_Engineering/data-engineering-main/01-spark/data/tripdata_2021_1-7.csv
+---+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+--

In [8]:
data.createOrReplaceTempView("mobility_data")
# 해당 df를 SQL에서 사용하기 위해서 temporaryView에 등록 

In [9]:
spark.sql("select * from mobility_data limit 5").show()

22/10/04 20:31:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, airport_fee
 Schema: _c0, VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag, PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, airport_fee
Expected: _c0 but found: 
CSV file: file:///Users/sig6774/Desktop/Data_Engineering/data-engineering-main/01-spark/data/tripdata_2021_1-7.csv
+---+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+--

In [11]:
# 날짜와 시간을 분리
spark.sql("select split(tpep_pickup_datetime, ' ') from mobility_data").show()
# 공백을 기준으로 데이터를 분리 
# 원하는 컬럼을 split()안에 넣고 원하는 분리 조건을 작성하면 됨 

+----------------------------------+
|split(tpep_pickup_datetime,  , -1)|
+----------------------------------+
|              [2021-01-01, 00:3...|
|              [2021-01-01, 00:5...|
|              [2021-01-01, 00:4...|
|              [2021-01-01, 00:1...|
|              [2021-01-01, 00:3...|
|              [2021-01-01, 00:1...|
|              [2021-01-01, 00:0...|
|              [2021-01-01, 00:1...|
|              [2021-01-01, 00:3...|
|              [2021-01-01, 00:2...|
|              [2021-01-01, 00:1...|
|              [2021-01-01, 00:4...|
|              [2021-01-01, 00:1...|
|              [2021-01-01, 00:3...|
|              [2021-01-01, 00:4...|
|              [2021-01-01, 00:1...|
|              [2021-01-01, 00:3...|
|              [2021-01-01, 00:4...|
|              [2021-01-01, 00:0...|
|              [2021-01-01, 00:1...|
+----------------------------------+
only showing top 20 rows



In [13]:
spark.sql("select split(tpep_pickup_datetime, ' ')[0] as pickup_date from mobility_data").show()
# 공백을 기준으로 데이터를 분리 
# 원하는 컬럼을 split()안에 넣고 원하는 분리 조건을 작성하면 됨 
# as로 컬럼명 변경 

+-----------+
|pickup_date|
+-----------+
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
| 2021-01-01|
+-----------+
only showing top 20 rows



In [17]:
spark.sql("select pickup_date, count(*) as trips from (select split(tpep_pickup_datetime, ' ')[0] as pickup_date from mobility_data) group by pickup_date").show()
# 공백을 기준으로 나눠진 데이터를 기준으로 그룹을 나누고 해당 날짜의 value의 count를 세줌 

+-----------+-----+
|pickup_date|trips|
+-----------+-----+
| 2021-01-15|53024|
| 2021-01-12|50133|
| 2021-01-11|46849|
| 2021-01-20|49437|
| 2021-01-24|31033|
| 2021-01-10|29855|
| 2021-01-13|51306|
| 2021-02-22|50700|
| 2009-01-01|  111|
| 2021-01-28|56444|
| 2021-01-26|47818|
| 2021-01-27|52655|
| 2021-01-22|54285|
| 2021-01-21|53280|
| 2021-01-25|47423|
| 2021-01-03|26345|
| 2021-02-01| 6806|
| 2021-01-08|50477|
| 2021-01-19|51149|
| 2021-01-14|53051|
+-----------+-----+
only showing top 20 rows

